Tobias Kuhlmann, Karlsruhe Institute of Technology (KIT), tobias.kuhlmann@student.kit.edu

In [1]:
%matplotlib inline

# Pretty Display of Variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Double resolution plotting for retina display
%config InlineBackend.figure_format ='retina'

import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt
import random
import time

import glob
import os

import datetime as dt

## Option implied betas following Buss and Vilkov (2012)
Implements calculation of option implied correlations following Buss, A. and Vilkov, G., 2012. Measuring equity risk with option-implied correlations. The Review of Financial Studies, 25(10), pp.3113-3140.


## Idea

Buss and Vilkov use forward-looking information from option prices to estimate option-implied correlations and to construct an option-implied predictor of factor betas. All of that under risk neutral probability measure.

##### Intuition: 
- Only using historical information implies that the future is sufficiently similar to the past, which is questionable in financial markets -> Need for option implied information as they are forward-looking and traded, which means they include current market expectations. Use of option implied information may improve predictive quality
- Variance risk premium: Overinsurance of volatility risk -> Implied volatility is higher in magnitude than realized volatility
 - Intuition: Investors are also willing to pay a risk premium to hedge against changes in variance (rising variance)
- Correlation risk premium: Premium for uncertainty of future correlation. Need to insure future correlation, as a rise in correlation reduces diversification benefits. Underlying uncertainty: Correlations go crazy in crises -> Not much diversification in large market downturns and systemic crises, except with short positions. Result is that option-implied correlations are higher than realized.
 - Intuition: Investors are also willing to pay a risk premium to hedge against changes in correlations
 
##### Rep: Risk neutral vs realized
- Option implied measures are under risk neutral probability measure, which differs from realized / objective probability measure because of variance and correlation risk premium. Risk premia makes risk neutral moments biased predictors of realized moments. 
- Option implied correlations not directly observable, needs modeling choice -> parametric with assumptions

##### Technical conditions of correlation matrix
1. Propose a parametric way to estimate implied correlations not exceeding one
2. correlation risk premium is negative (so consistent with the literature)
3. correlation risk premium is higher in magnitude for low or negatively correlated stocks that are exposed to a higher risk of losing diversification benefits (so consistent with literature)
4. Correlation matrix should be consistent with empirical observations
    - Implied correlation is higher than realized
    - Correlation risk premium is larger in magnitude for pairs of stocks that provide higher diversification benefits (negative and low correlation)

## Theoretical implementation steps

##### Goal
- Implied correlation matrix $\Gamma^{Q}_{t}$ with elements $\rho^{Q}_{ij,t}$
- Implied betas $\beta^{Q}_{iM,t}$ calculated from implied correlations

##### Implied variance of market index
Observed implied variance of market index 
$$(\sigma^{Q}_{M,t})^2 = \sum_{i=1}^{N}\sum_{j=1}^{N}{w_iw_j\sigma^{Q}_{i,t}\sigma^{Q}_{j,t}\rho^{Q}_{ij,t}}$$
 
where $\sigma^{Q}_{i,t}$ is implied volatility of stock i in the index and $w_i$ are index weights
 
##### Parametric form of implied correlations $\rho^{Q}_{ij,t}$
$$\rho^{Q}_{ij,t} = \rho^{P}_{ij,t} - \alpha_t(1-\rho^{P}_{ij,t})$$
 
where $\rho^{P}_{ij,t}$ is expected correlation under objective measure (realized), $\alpha_t$ needs to be identified

##### Identify $\alpha_t$ in closed form
$$\alpha_t = \frac{(\sigma^{Q}_{M,t})^2-\sum_{i=1}^{N}\sum_{j=1}^{N}{w_iw_j\sigma^{Q}_{i,t}\sigma^{Q}_{j,t}\rho^{P}_{ij,t}}}{\sum_{i=1}^{N}\sum_{j=1}^{N}{w_iw_j\sigma^{Q}_{i,t}\sigma^{Q}_{j,t}(1-\rho^{P}_{ij,t})}}$$

To satisfy above conditions: $-1\leq\alpha_t\leq0$

##### Calculate betas $\beta_{iM,t}$ with option implied correlation
$$\beta^{Q}_{iM,t} = \frac{\sigma^{Q}_{i,t}\sum_{j=1}^{N}{w_j\sigma^{Q}_{j,t}\rho^{Q}_{ij,t}}}{(\sigma^{Q}_{M,t})^2}$$

For a multifactor model with multiple non-correlated factors, the beta $\beta_{ik,t}$, where $i$ is the stock and $k$ the factor, can be calulated as the ratio of stock-to-factor covariance $\sigma_{ik,t}$ to the factor variance $\sigma^{2}_{k,t}$. $\rho_{ij,t}$ is the pairwise stock correlation, $\sigma_{j,t}$ stock volatilities, and $w_j$ the factor-mimicking portfolio weights.
 
 
 

## Data

##### Data needed
- Return time series to calculate rolling window stock-stock correlations $\rho^{P}_{ij,t}$
- Stock weights in index over time 
- Implied volatilities of every single stock i $\sigma^{Q}_{i,t}$
- Implied volatilities of Index $\sigma^{Q}_{M,t}$
- Use stock-stock correlations, implied volatilities of single stocks and index implied volatility to calculate $\alpha_t$


## Data Simulation

Simulated return and volatility data

In [2]:
n_constituents = 500
t = 4000
# 500 stock return series
single_stock_returns = np.random.rand(t, n_constituents)
# weight vectors for every t
w = np.random.rand(t, n_constituents)
w[1,:].shape # specific t
# implied vol sigma vector of all single stocks for every t
sigma_single_stocks = np.random.rand(t, n_constituents)
sigma_single_stocks.shape
sigma_single_stocks[1,:].shape # specific t
# implied vol sigma of index for every t
sigma_index = np.random.rand(t, 1)

(500,)

(4000, 500)

(500,)

##### Rolling window stock-stock correlations $\rho^{P}_{ij,t}$ under empirical measure P
- rolling window correlations of log returns: rolling window = 252

In [3]:
def log_returns(x):
    """
    Calculates log returns
    
    Parameters
    ----------
    x : array_like, shape (time, n_constituents)
        DataFrame with constituents in columns and timeseries values in rows
    Returns
    -------
    log_returns : ndarray, shape (time, n_constituents)
        Returns log returns for every constituent
    """
    # convert to pandas DataFrame calc log returns
    single_stock_returns = pd.DataFrame(x)
    # calc log returns
    return np.log(single_stock_returns / single_stock_returns.shift(1, axis=0))

def rolling_window_corr(df, rolling_window=252):
    """
    Calculates rolling window correlation matrices between constituents for every time point
    
    Parameters
    ----------
    df : array_like, shape (time, n_constituents)
        DataFrame with constituents in columns and timeseries values in rows
    rolling_window: array_like, scalar, shape (1,1)
        Number of time steps to use for rolling window. Default 252 for yearly
    Returns
    -------
    rolling_corrs : ndarray, shape (time, n_constituents, n_constituents)
        Returns correlation matrices for every time step in 3D structure 
    """
    # rolling window correlations
    rolling_corrs = df.rolling(window=rolling_window, axis=0).corr(pairwise=True)
    # convert to numpy 3D ndarray, shape (time, constituents, constituents)
    rolling_corrs_3D = rolling_corrs.values.reshape(
                                    (-1, single_stock_returns.shape[1],single_stock_returns.shape[1]))
    return rolling_corrs_3D

##### Correlation calculation small simulation test

In [4]:
# problem: calculating 4000 500x500 correlation matrices takes some time
# start timing
t0 = time.time()
# log returns
constituents_log_returns = log_returns(x=single_stock_returns)

# rolling window correlations
rolling_corrs_3D = rolling_window_corr(df=constituents_log_returns, rolling_window=252)
rolling_corrs_3D.shape

# emd timing
t1 = time.time()
total = t1-t0
print(f"correlation calculation execution time: {total} seconds")

(4000, 500, 500)

correlation calculation execution time: 1142.7020871639252 seconds


In [10]:
def strip_rolling_window_nan(sigma_q_m, w, sigma, roh):
    """
    Strips nan resulting from rolling window correlation calculation
    
    Parameters
    ----------
    sigma_q_m: scalar or array_like, shape (1, 1)
        Index implied volatility at time t
    w : array_like, shape (n_constituents, 1)
        Weight vector of all index constituents at time t.
    sigma: array_like, shape (n_constituents, 1)
        Volatility vector of all index constituents at time t.
    roh: array_like, shape (n_constituents, n_constituents)
        Correlation matrix of all index constituents at time t.
    Returns
    -------
    sigma_q_m: scalar or array_like, shape (1, 1)
        Stripped nan: Index implied volatility at time t
    w : array_like, shape (n_constituents, 1)
        Stripped nan: Weight vector of all index constituents at time t.
    sigma: array_like, shape (n_constituents, 1)
        Stripped nan: Volatility vector of all index constituents at time t.
    roh: array_like, shape (n_constituents, n_constituents)
        Stripped nan: Correlation matrix of all index constituents at time t.
    """
    
    return sigma_q_m, w, sigma, roh

## Implied correlation estimation

##### Approach
- Return time series to calculate stock-stock correlations $\rho^{P}_{ij,t}$
- Stock weights $w_i$ in index over time (time-varying or time-invariant?)
- Implied volatilities of every single stock i $\sigma^{Q}_{i,t}$
- Implied volatilities of Index $\sigma^{Q}_{M,t}$
- Use stock-stock correlations, implied volatilities of single stocks and index implied volatility to calculate $\alpha_t$

Function sum_of_sum calculates this part of $alpha_t$ formula:
$$\sum_{i=1}^{N}\sum_{j=1}^{N}{w_iw_j\sigma^{Q}_{i,t}\sigma^{Q}_{j,t}\rho^{P}_{ij,t}}$$

In [6]:
def sum_of_sum(w, sigma, roh):
    """
    Calculates sum of sum with more efficient matrix build in multiplication
    
    Parameters
    ----------
    w : array_like, shape (n_constituents, 1)
        Weight vector of all index constituents at time t.
    sigma: array_like, shape (n_constituents, 1)
        Volatility vector of all index constituents at time t.
    roh: array_like, shape (n_constituents, n_constituents)
        Correlation matrix of all index constituents at time t.
    Returns
    -------
    sumsum : scalar, shape (1,1)
        Returns sum of sum.
    """
    # outer product with matrix multiplication
    w_outer_prod = w @ w.T
    sigma_outer_prod = sigma @ sigma.T

    # element wise multiplication
    prod_prod = w_outer_prod * sigma_outer_prod * roh
    # sum all elements
    return sum(sum(prod_prod))



Function sum_of_sum calculates $alpha_t$ formula:
$$\alpha_t = \frac{(\sigma^{Q}_{M,t})^2-\sum_{i=1}^{N}\sum_{j=1}^{N}{w_iw_j\sigma^{Q}_{i,t}\sigma^{Q}_{j,t}\rho^{P}_{ij,t}}}{\sum_{i=1}^{N}\sum_{j=1}^{N}{w_iw_j\sigma^{Q}_{i,t}\sigma^{Q}_{j,t}(1-\rho^{P}_{ij,t})}}$$

In [7]:
def calc_alpha_t(sigma_q_m, w, sigma, roh):
    """
    Calculates alpha_t 
    
    Parameters
    ----------
    sigma_q_m: scalar or array_like, shape (1, 1)
        Index implied volatility at time t
    w : array_like, shape (n_constituents, 1)
        Weight vector of all index constituents at time t.
    sigma: array_like, shape (n_constituents, 1)
        Volatility vector of all index constituents at time t.
    roh: array_like, shape (n_constituents, n_constituents)
        Correlation matrix of all index constituents at time t.
    Returns
    -------
    alpha_t : scalar, shape (1,1)
        Returns alpha at time t.
    """
    sum1 = sum_of_sum(w=w, sigma=sigma, roh=roh)
    sum2 = sum_of_sum(w=w, sigma=sigma, roh=1-roh)
    
    alpha_t = (sigma_q_m - sum1) / sum2
    
    return alpha_t

Function q_correlation calculates $\rho^{Q}_{ij,t}$ formula:
$$\rho^{Q}_{ij,t} = \rho^{P}_{ij,t} - \alpha_t(1-\rho^{P}_{ij,t})$$

In [8]:
def calc_q_correlation(roh_p_t, alpha_t):
    """
    Calculates roh_q_t, risk neutral correlation at time t
    
    Parameters
    ----------
    roh_p_t : array_like, shape (n_constituents, n_constituents)
        P correlation under empirical measure at time t.
    alpha_t: scalar or array_like, shape (1, 1)
        alpha scaling value at time t
    Returns
    -------
    sumsum : array_like, shape (n_constituents, n_constituents)
        Correlation matrix under Q, risk neutral measure, at time t.
    """
    return roh_p_t - alpha_t * (1 - roh_p_t)

Function q_beta calculates betas $\beta_{ik,t}$ with option implied correlation
$$\beta^{Q}_{iM,t} = \frac{\sigma^{Q}_{i,t}\sum_{j=1}^{N}{w_j\sigma^{Q}_{j,t}\rho^{Q}_{ij,t}}}{(\sigma^{Q}_{M,t})^2}$$


In [9]:
def q_beta(sigma_q_m, w, sigma, roh):
    """
    Calculates Q betas under risk neutral measure
    
    Parameters
    ----------
    sigma_q_m: scalar or array_like, shape (1, 1)
        Index implied volatility at time t
    w : array_like, shape (n_constituents, 1)
        Weight vector of all index constituents at time t.
    sigma: array_like, shape (n_constituents, 1)
        Volatility vector of all index constituents at time t.
    roh: array_like, shape (n_constituents, n_constituents)
        Correlation matrix of all index constituents at time t.
    Returns
    -------
    q_beta : scalar, shape (n_constituents, 1)
        Returns Q beta for every constituent under risk neutral measure at time t.
    """
    
    
    return beta_i_t

##### Test functions
Test data for validation

In [90]:
# test case for method
sigma_q_m = 0.1
w = np.array([[1/6,1/6,2/3]]).T
print(f"w shape: {w.shape}")
sigma = np.array([[0.1,0.2,0.3]]).T
print(f"sigma shape: {sigma.shape}")
roh = np.array([[1,0.1,0.1], [0.1,1,0.1], [0.1,0.1,1]])
print(f"roh shape: {roh.shape}")
# assert: shapes of arrays

# assert: result should be 0.0435
print(f"sumsum: {sum_of_sum(w=w, sigma=sigma, roh=roh)}")

# assert: result should be 2.9737
alpha_t = calc_alpha_t(sigma_q_m=sigma_q_m, w=w, sigma=sigma, roh=roh)
print(f"alpha_t: {alpha_t}")

# assert: result should be [[ 1.        , -2.57631579, -2.57631579],
#       [-2.57631579,  1.        , -2.57631579],
#       [-2.57631579, -2.57631579,  1.        ]]
q_correlation = calc_q_correlation(roh_p_t = roh, alpha_t=alpha_t)
print(f"q_correlation: {q_correlation}")

w shape: (3, 1)
sigma shape: (3, 1)
roh shape: (3, 3)
sumsum: 0.0435
alpha_t: 2.973684210526316
q_correlation: [[ 1.         -2.57631579 -2.57631579]
 [-2.57631579  1.         -2.57631579]
 [-2.57631579 -2.57631579  1.        ]]


##### Test computation effort for whole simulated dataset

In [245]:
# weight vectors for every t
w[0,:].shape # specific t
# implied vol sigma vector of all single stocks for every 
sigma_single_stocks.shape
sigma_single_stocks[0,:].shape # specific t
# implied vol sigma of index for every t
sigma_index.shape
# correlation matrix
rolling_corrs_3D.shape
rolling_corrs_3D[0,:,:].shape

(500,)

(4000, 500)

(500,)

(4000, 1)

(4000, 500, 500)

(500, 500)

In [ ]:
np.isnan(rolling_corrs_3D[250:255])

In [246]:
# one t
alpha_t = calc_alpha_t(sigma_q_m=sigma_index[0], w=w[0,:], sigma=sigma_single_stocks[0,:], roh=rolling_corrs_3D[0,:,:])
print(f"alpha_t: {alpha_t}")

q_correlation = calc_q_correlation(roh_p_t = rolling_corrs_3D[0,:,:], alpha_t=alpha_t)
print(f"q_correlation: {q_correlation}")

alpha_t: [nan]
q_correlation: [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
